## NB very simple model for testing

In [18]:
%pip install xgboost

Active code page: 1252
Note: you may need to restart the kernel to use updated packages.


In [19]:
%matplotlib inline
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error


In [20]:
data = pd.read_csv('final.csv')

In [21]:
data.head(8)

,Date,Time,Player_count
0,9/24/2023,0:00,2711248
1,9/24/2023,0:05,2726267
2,9/24/2023,0:10,2712687
3,9/24/2023,0:15,2745816
4,9/24/2023,0:20,2756843
5,9/24/2023,0:25,2768107
6,9/24/2023,0:30,2777797
7,9/24/2023,0:35,2786147


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          3755 non-null   object
 1   Time          3755 non-null   object
 2   Player_count  3755 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 88.1+ KB


In [23]:
data = data.drop(data[(data["Player_count"]== 0)].index)

In [24]:
data["New_date"] = pd.to_datetime(data['Date'], format='%m/%d/%Y')

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3606 entries, 0 to 3754
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3606 non-null   object        
 1   Time          3606 non-null   object        
 2   Player_count  3606 non-null   int64         
 3   New_date      3606 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 140.9+ KB


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3606 entries, 0 to 3754
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          3606 non-null   object        
 1   Time          3606 non-null   object        
 2   Player_count  3606 non-null   int64         
 3   New_date      3606 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 140.9+ KB


### Feature engineering (transform data to be used for the machine learning algo) 

In [27]:

data["New_time"] = pd.to_datetime(data['Time'], format='%H:%M')

In [28]:
data["year"] = data["New_date"].dt.year
data["month"] = data["New_date"].dt.month
data["day"] = data["New_date"].dt.day
data["hour"] = data["New_time"].dt.hour
data["min"] = data["New_time"].dt.minute

In [29]:
data.drop(columns=["Date","Time","New_date","New_time"], inplace=True)

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3606 entries, 0 to 3754
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Player_count  3606 non-null   int64
 1   year          3606 non-null   int64
 2   month         3606 non-null   int64
 3   day           3606 non-null   int64
 4   hour          3606 non-null   int64
 5   min           3606 non-null   int64
dtypes: int64(6)
memory usage: 197.2 KB


In [31]:
data.head()

,Player_count,year,month,day,hour,min
0,2711248,2023,9,24,0,0
1,2726267,2023,9,24,0,5
2,2712687,2023,9,24,0,10
3,2745816,2023,9,24,0,15
4,2756843,2023,9,24,0,20


### One hot encoding

### Preapering for training 

### Models and testing

#### xgboost

In [32]:
tss = TimeSeriesSplit(n_splits=5, test_size=11, gap=11)
df = data.sort_index()


fold = 0
preds = []
scores = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    FEATURES = ['year', 'month', 'day', 'hour', 'min']
    TARGET = 'Player_count'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=50,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.01)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = np.sqrt(mean_squared_error(y_test, y_pred))
    scores.append(score)

[0]	validation_0-rmse:1755891.00177	validation_1-rmse:2523757.63771
[100]	validation_0-rmse:692442.83417	validation_1-rmse:1188393.30562
[200]	validation_0-rmse:328761.13229	validation_1-rmse:673221.07893
[300]	validation_0-rmse:212850.41783	validation_1-rmse:394853.85739


c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [03:30:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:167324.73106	validation_1-rmse:315610.68452


[500]	validation_0-rmse:144080.32090	validation_1-rmse:251400.59437
[600]	validation_0-rmse:128392.75038	validation_1-rmse:202173.49539
[700]	validation_0-rmse:118013.85830	validation_1-rmse:167056.83433
[800]	validation_0-rmse:110645.42035	validation_1-rmse:138714.75588
[900]	validation_0-rmse:105461.34231	validation_1-rmse:122013.44319
[999]	validation_0-rmse:101020.71212	validation_1-rmse:108608.92195
[0]	validation_0-rmse:1759899.18610	validation_1-rmse:2337510.96722
[100]	validation_0-rmse:694605.19080	validation_1-rmse:977049.37694
[200]	validation_0-rmse:330646.36790	validation_1-rmse:438950.55034
[300]	validation_0-rmse:214235.76404	validation_1-rmse:142009.06173
[400]	validation_0-rmse:168163.85490	validation_1-rmse:60049.54139


c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [03:30:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


...

[400]	validation_0-rmse:168344.32067	validation_1-rmse:132327.57953
[500]	validation_0-rmse:145570.76987	validation_1-rmse:73183.63783
[600]	validation_0-rmse:129918.31189	validation_1-rmse:64982.57692
[614]	validation_0-rmse:128195.03042	validation_1-rmse:67555.37181
[0]	validation_0-rmse:1764796.67694	validation_1-rmse:2925794.40547
[100]	validation_0-rmse:697024.76886	validation_1-rmse:1364886.91514
[200]	validation_0-rmse:331321.81371	validation_1-rmse:678962.36314
[300]	validation_0-rmse:214049.44184	validation_1-rmse:344775.70925
[400]	validation_0-rmse:168519.06602	validation_1-rmse:224612.11990


c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [03:30:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[500]	validation_0-rmse:145761.71394	validation_1-rmse:130262.26115
[600]	validation_0-rmse:130448.76574	validation_1-rmse:67357.71155
[700]	validation_0-rmse:120550.13767	validation_1-rmse:35493.38280
[755]	validation_0-rmse:115921.09477	validation_1-rmse:40914.38719
[0]	validation_0-rmse:1767771.96906	validation_1-rmse:3105092.09726
[100]	validation_0-rmse:698165.54292	validation_1-rmse:1265338.78964
[200]	validation_0-rmse:331669.34456	validation_1-rmse:582271.38736
[300]	validation_0-rmse:213845.42739	validation_1-rmse:292886.13203
[400]	validation_0-rmse:168366.59743	validation_1-rmse:169780.72950


c:\Users\Tommy\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [03:30:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[500]	validation_0-rmse:145899.98016	validation_1-rmse:92580.82607
[600]	validation_0-rmse:130350.26398	validation_1-rmse:36465.99526
[700]	validation_0-rmse:120296.46737	validation_1-rmse:21783.44280
[724]	validation_0-rmse:118251.94821	validation_1-rmse:27413.78558


In [33]:
print(f'Score across folds {np.mean(scores):0.4f}')
print(f'Fold scores:{scores}')

Score across folds 51395.3164
Fold scores:[108568.87661209588, 33429.3839504532, 61137.55498882306, 34721.963745294495, 19118.80253373269]


Saving model to pkl file for later use


In [34]:
reg.save_model("reg_model.json")
